# Graph Generation Scripts

Edit the variables below and run the cells in order to generate graph files that can be opened in Gephi

In [1]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/cb/a1/c698cf319e9cfed6b17376281bd0efc6bfc8465698f54170ef60a485ab5d/beautifulsoup4-4.8.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/05/27e1466475e816d3001efb6e0a85a819be17411420494a1e602c36f8299d/bleach-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8f/b7/f329cfdc75f3d28d12c65980e4469e2fa373f1953f5df6e370e84ea2e875/decorator-4.4.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/95/38152ca47b749553d2ff93fe2ad6cbe9d724ced152c0a48b5ba2a13a3f83/geomdl-5.2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/31/74dcb59a601b95fce3b0334e8fc9db758f78e43075f22aeb3677dfb19f4c/importlib_metadata-1.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/92/8fec943b5b81078399f969f00557804d884c96fcd0bc296e81a2ed4fd270/ipykernel-5.1.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/

In [6]:
# 1: Enter the input file name

input_file_name = '504b_The Voyage of the Dawn Treader - C. S. Lewis - complete.xml'

In [8]:
# 2: Generate output file names

output_root = input_file_name[:-4]

complete_output_file_name = output_root +'-complete.gexf'
syuzhet_output_file_name = output_root + '-syuzhet.gexf'
topoi_output_file_name = output_root + '-topoi-only.gexf'
topoi_with_time_output_file_name = output_root + '-topoi-with-time.gexf'
chronotope_output_file_name = output_root + '-frames-and-chronotopic-archetypes.gexf'
chronotope_II_output_file_name = output_root + '-deep-chronotopes.gexf'
chronotope_III_output_file_name = output_root + '-chronotopic-archetypes-and-toporefs.gexf'

In [3]:
# 3: Import libraries

from lxml import etree
import networkx as nx

from bs4 import BeautifulSoup, element

In [4]:
chronotopes = [
    'anti-idyll',
    'castle',
    'distortion',
    'encounter',
    'idyll',
    'metanarrative',
    'parlour',
    'public square',
    'road',
    'threshold',
    'provincial town',
    'wilderness'
]

connections = [
    'direct',
    'indirect',
    'interrupt',
    'jump',
    'charshift',
    'projection',
    'metatextual',
    'paratextual',
    'intratextual',
    'metaphor'
]
        

import pprint
        
def check_xml(xml_element):
    """
    Checks for the most common problems with a Chrono-Carto-encoded XML file. 
    WARNING: this doesn't pick up inconsistencies in the naming of topoi, source and target tags
    """
    
    
    errors = {'nodes': [], 'connections': [], 'toporefs': []}
    
    warnings = {'nodes': [], 'connections': [], 'toporefs': []}
    
    sources_and_targets = {'sources': [], 'targets': []}
    
    nodes = []
    
    for topos in xml_element.iter('topos'):
        try:
            topos.attrib['type']
            if topos.attrib['type'] not in chronotopes:
                warnings['nodes'].append(['Check node type attribute on line ' + str(topos.sourceline), topos.attrib])
        except:
            errors['nodes'].append(['No type attribute on line ' + str(topos.sourceline), topos.attrib])
        try:
            topos.attrib['framename']
            nodes.append(topos.attrib['framename'])
        except:
            errors['nodes'].append(['No framename attribute on line ' + str(topos.sourceline), topos.attrib])

            
    for connection in xml_element.iter('connection'):
        try:
            connection.attrib['source']
        except:
            errors['connections'].append(['No source attribute on line ' + str(connection.sourceline), connection.attrib])
        try:
            connection.attrib['target']
        except:
            errors['connections'].append(['No target attribute on line ' + str(connection.sourceline), connection.attrib])
        try:
            connection.attrib['relation']
            if connection.attrib['relation'] not in connections:
                warnings['connections'].append(['Check relation attribute on line ' + str(connection.sourceline), connection.attrib])
        except:
            errors['connections'].append(['No relation attribute on line ' + str(connection.sourceline), connection.attrib])

            
    for connection in xml_element.iter('connection'):
        try:
            if connection.attrib['source'] not in nodes:
                sources_and_targets['sources'].append(['No matching topos for source attribute "' + connection.attrib['source'] + '" on line ' + str(connection.sourceline), connection.attrib])
            if connection.attrib['target'] not in nodes:
                sources_and_targets['targets'].append(['No matching topos for target attribute "' + connection.attrib['target'] + '" on line ' + str(connection.sourceline), connection.attrib])
        except:
            print(connection.sourceline, connection.attrib)
    
    
    for toporef in xml_element.iter('toporef'):
        try:
            toporef.attrib['role']
        except:
            errors['toporefs'].append(['No role attribute on line ' + str(toporef.sourceline), toporef.attrib])
        try:
            toporef.attrib['relation']
            if toporef.attrib['relation'] not in connections:
                warnings['toporefs'].append(['Check relation attribute on line ' + str(toporef.sourceline), toporef.attrib])
        except:
            errors['toporefs'].append(['No relation attribute on line ' + str(toporef.sourceline), toporef.attrib])
    
    print('Errors:')
    if (len(errors['connections']) > 0) or (len(errors['nodes']) > 0) or (len(errors['toporefs']) > 0):
        pprint.pprint(errors)
    else:
        print('No errors found!')
    print('')
    print('Attribute typos:')
    if (len(warnings['connections']) > 0) or (len(warnings['nodes']) > 0) or (len(warnings['toporefs']) > 0):
        pprint.pprint(warnings)
    else:
        print('No attribute typos found!')
    print('')
    print('Source and Target mis-matches')
    if (len(sources_and_targets['sources']) > 0) or (len(sources_and_targets['targets']) > 0):
        pprint.pprint(sources_and_targets)
    else:
        print('No mismatches found!')

In [9]:
# 4: Read the XML file and create empty graph objects
tree = etree.parse('files/xml/' + input_file_name)
root = tree.getroot()
complete = nx.DiGraph()
syuzhet = nx.DiGraph()
topoi = nx.DiGraph()
chronotope = nx.Graph()
chronotope_ii = nx.DiGraph()

In [10]:
check_xml(root)

Errors:
No errors found!

Attribute typos:
No attribute typos found!

Source and Target mis-matches
No mismatches found!


In [11]:
# 5: Define the functions for creating the graphs

def complete_graph(xml_element, graph):
    """
    Takes an XML element marked up using CLAYE and returns a populated graph of the spatial nodes, 
    litonyms, and the implied physical, psychological, or sensory connections between them. 
    
    Params:
    xml_element: an lxml XML element
    graph: a NetworkX Graph() object - nx.Graph()

    """
    
    # Add all the litonyms and topoi as nodes and connections as edges first
    for toporef in xml_element.iter('toporef'):
        graph.add_node('"' + toporef.text + '"', node_type='toporef')
    
    topos_count = 0
    
    for topos in xml_element.iter('topos'):
        topos_count_str = str(topos_count)
        try:
            print(topos.attrib)
            framename = topos.attrib['framename']
            new_length = graph.nodes[framename]['length'] + len(''.join(topos.itertext()).strip())
            graph.nodes[framename]['length'] = new_length
            graph.nodes[framename]['timeframes'] = graph.nodes[framename]['timeframes'] + ',' + topos_count_str
            
        except KeyError:
            graph.add_node(topos.attrib['framename'], length=len(''.join(topos.itertext()).strip()), chronotope=topos.attrib['type'], node_type="topos", timeframes=topos_count_str)

        topos_count += 1
    
    for connection in xml_element.iter('connection'):
        try:
            graph.add_edge(connection.attrib['source'], connection.attrib['target'], relation=connection.attrib['relation'])
        except:
            graph.add_edge(connection.attrib['source'], connection.attrib['target'], relation='none')
    
    # Connect the toporefs to the containing topoi
    for toporef in xml_element.iter('toporef'):
        if ('sequence' in toporef.attrib.keys()):
            pass
        else:
            parent = toporef.getparent()
            containing_node = None
            if parent.tag == 'topos':
                containing_node = parent.attrib['framename']
            elif parent.tag == 'connection':
                containing_topos = parent.getparent()
                try:
                    containing_node = containing_topos.attrib['framename']
                except:
                    pass
            try:
                graph.add_edge(containing_node, '"' + toporef.text + '"', relation=toporef.attrib['relation'])
            except:
                graph.add_edge(containing_node, '"' + toporef.text + '"', relation='none')
    
    # Connect the toporef sequences to one another
    sequences = {}
    for toporef in xml_element.iter('toporef'):
        try:
            sequence = toporef.attrib['sequence']
            sequences[sequence] = []
        except:
            pass
        
    for sequence in sequences.keys():
        for toporef in xml_element.iter('toporef'):
            try:
                sequence = toporef.attrib['sequence']
                sequences[sequence].append(toporef)
            except:
                pass
    
    for sequence, toporef_list in sequences.items():
        prev_toporef = None
        
        for toporef in toporef_list:    
            if prev_toporef == None:
                parent = toporef.getparent()
                containing_node = None
                if parent.tag == 'topos':
                    containing_node = parent.attrib['framename']
                elif parent.tag == 'connection':
                    containing_topos = parent.getparent()
                    try:
                        containing_node = containing_topos.attrib['framename']
                    except:
                        pass
                try:
                    graph.add_edge(containing_node, '"' + toporef.text + '"', relation=toporef.attrib['relation'])
                except:
                    graph.add_edge(containing_node, '"' + toporef.text + '"', relation='none')
                
                prev_toporef = toporef
                    
            else:
                graph.add_edge('"' + prev_toporef.text + '"', '"' + toporef.text + '"', relation=toporef.attrib['relation'])
                prev_toporef = toporef
            
        


def syuzhet_graph(xml_element, graph):
    """
    Takes an XML element marked up using CLAYE and returns a populated graph of the spatial nodes
    connected sequentially as they appear in the text
    Corresponds (loosely) with the syuzhet or story order of the text.
    
    Params:
    xml_element: an lxml XML element
    graph: a NetworkX Graph() object - nx.Graph()

    """
    topoi = []

    for topos in xml_element.iter('topos'):
        topoi.append([topos.attrib['framename'], topos.attrib['type'], len(''.join(topos.itertext()).strip())])

    prev_node = None
    
    for t in topoi:
        if prev_node == None:
            prev_node = t[0]
            graph.add_node(t[0], chronotope=t[1], length=t[2])
        else:
            try:
                graph.nodes[t[0]]['length'] += t[2]
            except KeyError:
                graph.add_node(t[0], chronotope=t[1], length=t[2])
            
            connection = None
            
            for c in xml_element.iter('connection'):
                if (c.attrib['source'] == prev_node) and (c.attrib['target'] == t[0]):
                    graph.add_edge(prev_node, t[0], relation=c.attrib['relation'])
                    connection = [prev_node, t[0], c.attrib['relation']]
                    print(c.attrib)
                else:
                    #graph.add_edge(prev_node, t[0], relation='none')
                    connection = [prev_node, t[0], 'none']
            
            prev_node = t[0]

            
def topoi_graph(xml_element, graph):
    """
    Iterate over an XML element and its children and generate a graph of topoi nodes and connections, including attributes.
    xml_element: an eTree XML element
    graph: a NetworkX Graph() object - nx.Graph()
    """
    
    for topos in xml_element.iter('topos'):
        try:
            graph.nodes[topos.attrib['framename']]['length'] += len(''.join(topos.itertext()).strip())
        except KeyError:
            graph.add_node(topos.attrib['framename'], chronotope=topos.attrib['type'], length=len(''.join(topos.itertext()).strip()))

    for c in xml_element.iter('connection'):
        try:
            graph.add_edge(c.attrib['source'], c.attrib['target'], relation=c.attrib['relation'])
        except KeyError:
            print(c.attrib)
            

def topoi_graph_with_time(xml_element, graph):
    """
    Iterate over an XML element and its children and generate a graph of topoi nodes and connections, including attributes.
    xml_element: an eTree XML element
    graph: a NetworkX Graph() object - nx.Graph()
    """
    
    topos_count = 0
    
    for topos in xml_element.iter('topos'):
        topos_count_string = str(topos_count)
        try:
            graph.nodes[topos.attrib['framename']]['length'] += len(''.join(topos.itertext()).strip())
            timeframes = graph.nodes[topos.attrib['framename']]['timeframes'] + ',' + topos_count_string 
            graph.nodes[topos.attrib['framename']]['timeframes'] = timeframes
        except KeyError:
            graph.add_node(topos.attrib['framename'], chronotope=topos.attrib['type'], length=len(''.join(topos.itertext()).strip()), timeframes=topos_count_string)
            
        topos_count += 1
    

    for c in xml_element.iter('connection'):
        try:
            graph.add_edge(c.attrib['source'], c.attrib['target'], relation=c.attrib['relation'])
        except KeyError:
            print(c.attrib)



def chronotope_graph(xml_element, graph):
    """
    Takes an XML element marked up using CLAYE and returns a populated graph of the topoi
    and their associated chronotopes
    
    Params:
    xml_element: an lxml XML element
    graph: a NetworkX Graph() object - nx.Graph()
    """
    
    for topos in xml_element.iter('topos'):
        graph.add_node(topos.attrib['type'], node_type='chronotope')
        graph.add_node(topos.attrib['framename'], node_type='setting')
        graph.add_edge(topos.attrib['type'], topos.attrib['framename'])


def chronotope_graph_ii(soup, graph):
    
    # Add all the connections first
    edges = []
    for connection in soup.find_all('connection'):
        source = connection.get('source')
        target = connection.get('target')
        relation = connection.get('relation')
        
        source_chronotope = None
        target_chronotope = None
        
        if (connection.parent.get('framename') == source):
            source_chronotope = connection.parent.get('type')
        
        else:
            for el in connection.previous_elements:
                if (isinstance(el, element.Tag) == True):
                    if el.get('framename') == source and source_chronotope is None:
                        source_chronotope = el.get('type')
                    elif el.get('framename') == target and target_chronotope is None:
                        target_chronotope = el.get('type')
        
        for el in connection.next_elements:
            if (isinstance(el, element.Tag) == True and el.get('type') is not None):
                if el.get('framename') == target and target_chronotope is None:
                    target_chronotope = el.get('type')
                elif el.get('framename') == source and source_chronotope is None:
                    source_chronotope = el.get('type')
        
        if (source_chronotope != None and target_chronotope != None):
            edges.append((source_chronotope, target_chronotope, relation))
            
    edges = list(set(edges))
    
    for e in edges:
        graph.add_edge(e[0], e[1], relation=e[2])
        
    print(graph.nodes)
    
    # Then iterate over the topoi and calculate the number of characters in each, appending the values to the nodes
    chronotopes = {}
    for topos in soup.find_all('topos'):
        chronotope = topos.get('type')
        try:
            if chronotope not in chronotopes.keys():
                chronotopes[chronotope] = len(topos.get_text())
            else:
                chronotopes[chronotope] += len(topos.get_text())
        except:
            pass
    
    for c, attribs in chronotopes.items():
        print(c)
        graph.nodes[c]['length'] = attribs    

    
def chronotope_graph_iii(xml_element, graph):
    """
    Takes an XML element marked up using CLAYE and returns a populated graph of the chronotope archteypes, 
    their connections, and their associated toporefs
    
    """
    topoi = {}
    for topos in xml_element.iter('topos'):
        try: 
            chronotope = topos.attrib['type']
            graph.nodes[chronotope]['length'] += len(''.join(topos.itertext()).strip())
        except KeyError:
            graph.add_node(topos.attrib['type'], length=len(''.join(topos.itertext()).strip()))
        topoi[topos.attrib['framename']] = topos.attrib['type']
    
    for connection in xml_element.iter('connection'):
        try:
            source_chronotope = topoi[connection.attrib['source']]
            target_chronotope = topoi[connection.attrib['target']]
            relation = connection.attrib['relation']
            graph.add_edge(source_chronotope, target_chronotope, relation=relation)
        except:
            print('error')
    
    for topos in xml_element.iter('topos'):
        try:
            chronotope = topos.attrib['type']
            for toporef in topos.iter('toporef'):
                graph.add_edge(chronotope, '"' + toporef.text + '"', relation=toporef.attrib['relation'])
        except:
            pass

In [12]:
topoi_cc_over_time = nx.DiGraph()

def topoi_with_chronotopes_and_connections_over_time(xml_element, graph):
    """
    Takes an XML element and sequentially builds a populated graph of topoi, recording the temporal index, size, and chronotope 
    of each as they are encountered, and the temporal index of the connecting edges.
    """
    topoi = {}
    connections = []
    index = 0
    
    for el in xml_element.iter():
        if el.tag == 'topos':
            if el.attrib['framename'] not in topoi.keys():
                topoi[el.attrib['framename']] = {}
                topoi[el.attrib['framename']]['chronotopes'] = [el.attrib['type']]
                topoi[el.attrib['framename']]['time_indices'] = [str(index)]
                topoi[el.attrib['framename']]['text_lengths'] = [str(len(el.text))]
            else:
                topoi[el.attrib['framename']]['chronotopes'].append(el.attrib['type'])
                topoi[el.attrib['framename']]['time_indices'].append(str(index))
                topoi[el.attrib['framename']]['text_lengths'].append(str(len(el.text)))
                
            
            
        if el.tag == 'connection':
            connection = {}
            connection['source'] = el.attrib['source']
            connection['target'] = el.attrib['target']
            connection['relation'] = el.attrib['relation']
            connection['source_index'] = index 
            connection['target_index'] = index +1
            connections.append(connection)
            
            index = index +1
    
    for node, attributes in topoi.items():
        chronotopes = ', '.join(attributes['chronotopes'])
        indices = ', '.join(attributes['time_indices'])
        text_lengths = ', '.join(attributes['text_lengths'])
        graph.add_node(node, chronotopes=chronotopes, time_indices=indices, text_lengths=text_lengths)
    
    for edge in connections:
        graph.add_edge(edge['source'], edge['target'], source_index=edge['source_index'], target_index=edge['target_index'], relation=edge['relation'])   

        
topoi_with_chronotopes_and_connections_over_time(root, topoi_cc_over_time)

with open('files/graphs/pw_topoi_with_time.gexf', 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(topoi_cc_over_time):
        output_file.write(line)

In [37]:
check_xml(root)

Errors:
No errors found!

Attribute typos:
No attribute typos found!

Source and Target mis-matches
No mismatches found!


In [38]:
# 6: Complete

complete = nx.DiGraph()

complete_graph(root, complete)

with open('files/graphs/' + complete_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(complete):
        output_file.write(line)

{'type': 'metanarrative', 'framename': 'Preamble - Eustace Clarence Scrubb'}
{'type': 'threshold', 'framename': "The Scrubb's - Lucy's Room"}
{'framename': 'The Children Fall into the Sea', 'type': 'wilderness'}
{'framename': 'The Dawn Treader - Main Deck', 'type': 'public square'}
{'type': 'parlour', 'framename': 'The Dawn Treader - Stern Cabin'}
{'framename': 'The Dawn Treader - Main Deck', 'type': 'public square'}
{'framename': 'The Dawn Treader - Lower Deck', 'type': 'public square'}
{'framename': 'The Dawn Treader - Lower Deck Cabin', 'type': 'parlour'}
{'type': 'public square', 'framename': 'The Dawn Treader - Forecastle'}
{'type': 'castle', 'framename': 'The Dawn Treader - Fighting Top'}
{'type': 'public square', 'framename': 'The Dawn Treader - Poop Deck'}
{'type': 'parlour', 'framename': 'The Dawn Treader - Stern Cabin'}
{'framename': "Eustace's Diary", 'type': 'metanarrative'}
{'framename': 'The Dawn Treader - Stern Cabin', 'type': 'parlour'}
{'framename': 'The Dawn Treader -

In [39]:
# 7: Syujhet

syuzhet_graph(root, syuzhet)

with open('files/graphs/' + syuzhet_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(syuzhet):
        output_file.write(line)

{'source': 'Preamble - Eustace Clarence Scrubb', 'target': "The Scrubb's - Lucy's Room", 'relation': 'direct'}
{'source': "The Scrubb's - Lucy's Room", 'target': 'The Children Fall into the Sea', 'relation': 'direct'}
{'source': 'The Children Fall into the Sea', 'target': 'The Dawn Treader - Main Deck', 'relation': 'direct'}
{'source': 'The Dawn Treader - Main Deck', 'target': 'The Dawn Treader - Stern Cabin', 'relation': 'direct'}
{'source': 'The Dawn Treader - Stern Cabin', 'target': 'The Dawn Treader - Main Deck', 'relation': 'direct'}
{'source': 'The Dawn Treader - Main Deck', 'target': 'The Dawn Treader - Lower Deck', 'relation': 'direct'}
{'source': 'The Dawn Treader - Lower Deck', 'target': 'The Dawn Treader - Lower Deck Cabin', 'relation': 'direct'}
{'source': 'The Dawn Treader - Lower Deck Cabin', 'target': 'The Dawn Treader - Forecastle', 'relation': 'direct'}
{'source': 'The Dawn Treader - Lower Deck Cabin', 'target': 'The Dawn Treader - Forecastle', 'relation': 'jump'}
{'so

In [8]:
# 8: Topoi

topoi_graph(root, topoi)

with open('files/graphs/' + topoi_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(topoi):
        output_file.write(line)

In [13]:
tt = nx.DiGraph()

topoi_graph_with_time(root, tt)

with open('files/graphs/' + topoi_with_time_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(tt):
        output_file.write(line)

In [42]:
# 9: Chronotopes I

chronotope_graph(root, chronotope)

with open('files/graphs/' + chronotope_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(chronotope):
        output_file.write(line)

In [43]:
# 10: Chronotopes II
chronotope_ii = nx.DiGraph()

with open('files/xml/' + input_file_name) as fp:
    soup = BeautifulSoup(fp)    

chronotope_graph_ii(soup, chronotope_ii)

with open('files/graphs/' + chronotope_II_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(chronotope_ii):
        output_file.write(line)

['public square', 'road', 'provincial town', 'castle', 'metanarrative', 'anti-idyll', 'idyll', 'threshold', 'parlour', 'wilderness', 'encounter']
metanarrative
threshold
wilderness
public square
parlour
castle
encounter
provincial town
idyll
anti-idyll
road


In [29]:
# nx.write_gml(chronotope_ii, "test.gml")

chronotope_ii.edges()

OutEdgeView([('threshold', 'parlour'), ('threshold', 'anti-idyll'), ('threshold', 'idyll'), ('threshold', 'road'), ('threshold', 'threshold'), ('threshold', 'public square'), ('threshold', 'castle'), ('parlour', 'threshold'), ('parlour', 'public square'), ('parlour', 'idyll'), ('parlour', 'castle'), ('parlour', 'parlour'), ('parlour', 'road'), ('public square', 'threshold'), ('public square', 'public square'), ('castle', 'road'), ('castle', 'parlour'), ('castle', 'public square'), ('castle', 'threshold'), ('castle', 'encounter'), ('road', 'castle'), ('road', 'road'), ('road', 'parlour'), ('road', 'wilderness'), ('road', 'threshold'), ('wilderness', 'road'), ('anti-idyll', 'anti-idyll'), ('anti-idyll', 'road'), ('metanarrative', 'encounter'), ('metanarrative', 'metanarrative'), ('encounter', 'idyll'), ('idyll', 'idyll'), ('idyll', 'castle'), ('idyll', 'threshold'), ('idyll', 'metanarrative'), ('idyll', 'parlour')])

In [30]:
list(chronotope_ii.nodes())

['threshold',
 'parlour',
 'public square',
 'castle',
 'road',
 'wilderness',
 'anti-idyll',
 'metanarrative',
 'encounter',
 'idyll']

In [44]:
# 11: Chronotopes III

chronotope_iii = nx.DiGraph()

chronotope_graph_iii(root, chronotope_iii)

with open('files/graphs/' + chronotope_III_output_file_name, 'w') as output_file:
    for line in nx.readwrite.gexf.generate_gexf(chronotope_iii):
        output_file.write(line)